In [20]:
pip install pywavelets

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import random
import math
import time
import torch.nn.functional as F

from tqdm.notebook import tqdm
from torch import optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
import numpy as np
from  scipy import stats
import scipy
np.random.seed(1)
torch.manual_seed(1)
random.seed(1)

In [22]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [23]:
import os
os.chdir("/content/drive/MyDrive/MADE/Project/deap")

In [24]:
class Args:
  def __init__(self): #(data_path, epoch, batch_siz, image_size, learning_rate, weight_deca, learning_rate, learning_rate_gamma, weight_bce, load, output_dir)
    self.data_path = "/content/drive/MyDrive/MADE/semester2/CV/contest02/data/"
    self.epochs = 2
    self.batch_size = 100
    self.lr= 3e-4
    self.weight_decay= 1e-6
    self.learning_rate=None
    self.learning_rate_gamma=None
    self.weight_bce=1
    self.load=None
    self.output_dir="/content/drive/MyDrive/MADE/Project/lstm_models/"
    self.data_dir ="./data_preprocessed_python/"# "/content/drive/MyDrive/MADE/Project/train/physionet.org/"
args = Args()   

In [25]:
def get_feature_extractor():
       conv1 = nn.Conv2d(1, 8, kernel_size=(32, 1))
       relu1 = nn.ReLU()
       pool1 = nn.AvgPool2d((1, 2))
       conv2 = nn.Conv2d(8, 16, kernel_size=(1, 1))
       relu2 = nn.ReLU()
       pool2 = nn.AvgPool2d((1, 2))
       flat = nn.Flatten(1, 3)
       return (torch.nn.Sequential(conv1, relu1, pool1, conv2, relu2, pool2, flat))
def get_cell_classifier(input_size, koeff1, koeff2):
       drop1 = nn.Dropout(koeff1)
       fc1 = nn.Linear(128, 256)
       drop2 = nn.Dropout(koeff2)
       fc2 = nn.Linear(256, 2)
       return (torch.nn.Sequential(drop1, fc1, drop2, fc2))

class EmotionNet(torch.nn.Module): 
   def __init__(self, hidden_size, koeff1, koeff2, len_seq):
       super().__init__()
       #self.feature_extractor_list = nn.ModuleList([get_feature_extractor()] * len_seq)
       self.feature_extractor = get_feature_extractor()
       self.rnn = nn.LSTM(128, hidden_size, bidirectional = False, batch_first = True)
       #self.classifiers_list = nn.ModuleList([get_cell_classifier(hidden_size, koeff1, koeff2)] * len_seq)
       self.classifier = get_cell_classifier(hidden_size, koeff1, koeff2)
       self.soft = nn.LogSoftmax(dim = 2)
       
   def forward(self, src):
      #src = [batch size, seq_len, size1, size2]
      #print(f"src = {src.shape}")
      src = src.permute(1, 0, 2, 3)
      seq_len = src.shape[0]

      outputs = []
      #for i, fe in enumerate(self.feature_extractor_list):
      #      temp = fe(src[i].unsqueeze(1)).unsqueeze(0)
      #      print(temp.shape)
      #      outputs.append(temp)
      for i in range(seq_len):
             temp = self.feature_extractor(src[i].unsqueeze(1)).unsqueeze(0)
             #print(temp.shape)
             outputs.append(temp)
      

      outputs = torch.vstack(outputs)
      #print(f"outputs = {outputs.shape}")            
      outputs = outputs.permute(1, 0, 2)
      lstm_outputs, (hidden, c) = self.rnn(outputs)              
      #print(lstm_outputs.shape)
      outputs = lstm_outputs
      #outputs = outputs.permute(1, 0, 2)
      #classes = []
      #for i, classifier in enumerate(self.classifier_list):
      #      temp = classifier(outputs[i]).unsqueeze(0)
      #      print(temp.shape)
      #      classes.append(temp)
      #for i  in  range(seq_len):
      #      temp = self.classifier(outputs[i]).unsqueeze(0)
      #      #print(temp.shape)
      #      classes.append(temp)
      #print(classes)      
      outputs = self.classifier(outputs)
      outputs = self.soft(outputs).mean(axis = 1)
      print(outputs.shape)

      #classes = torch.vstack(classes)
      #print(f"classes = {classes.shape}")
      #classes = classes.permute(1, 0, 2)
      #print(f"classes = {classes.shape}")
      #classes = self.soft(classes)
      #print(classes)
      #classes_new = torch.zeros((classes.shape[0], 2)).to(device)
      #for i in range(classes.shape[0]):
          #print(f"classes[i] = {classes[i].shape}")
       #   classes_new[i] = classes[i].mean(axis = 0)
          
      #print(f"classes_new = {classes_new.shape}")
      
      
      return (outputs)

In [26]:
HIDDEN_SIZE = 128
DROPOUT_COEFF1 = 0.5
DROPOUT_COEFF2 = 0.5
LEN_SEQ = 60
def get_model(HIDDEN_SIZE, DROPOUT_COEFF1, DROPOUT_COEFF2, LEN_SEQ):
  model = EmotionNet(HIDDEN_SIZE, DROPOUT_COEFF1, DROPOUT_COEFF2, LEN_SEQ).to(device)
  return model


In [27]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = get_model(HIDDEN_SIZE, DROPOUT_COEFF1, DROPOUT_COEFF2, LEN_SEQ)
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

EmotionNet(
  (feature_extractor): Sequential(
    (0): Conv2d(1, 8, kernel_size=(32, 1), stride=(1, 1))
    (1): ReLU()
    (2): AvgPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0)
    (3): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
    (4): ReLU()
    (5): AvgPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0)
    (6): Flatten(start_dim=1, end_dim=3)
  )
  (rnn): LSTM(128, 128, batch_first=True)
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=128, out_features=256, bias=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=256, out_features=2, bias=True)
  )
  (soft): LogSoftmax(dim=2)
)

In [28]:
import glob
import pickle
from collections import Counter
import pywt



from collections import Counter
import scipy
from scipy import fft, ifft


class RandomAugmentation(object):
    def __init__(self, augmenters, probability):
        self._augmenters = augmenters
        self.probability = probability

    def __call__(self, sample):
        augmenter = random.choice(self._augmenters)
        #print(augmenter)
        if (augmenter):
            if (np.random.random() < self.probability):
               return augmenter(sample)
        return(sample)  


# class add_noise(object):
#     def __init__(self, ):
#         super(add_noise, self).__init__()

#     def __call__(self, sample: dict):
#         white_noise = np.random.normal(loc=0.0, scale=1.0, size=sample['data'].shape)
#         for i in range(len(sample['data']))
#             sample['data'][i] = sample['data'][i] + white_noise
#         return sample


class reset_part_in_time(object):
    def __init__(self, percentage):
        super(reset_part_in_time, self).__init__()
        self.percentage = percentage

    def __call__(self, sample: dict):
        len_data = len(sample['data'][0])
        interval_to_reset = int(len_data * self.percentage)
        for ncanal in range(NVIDEOS):
            begin = random.randint(0, len_data - interval_to_reset)
            sample['data'][ncanal, begin : begin + interval_to_reset] = 0

        return sample


class reset_part_in_freq(object):
    def __init__(self, percentage):
        super(reset_part_in_freq, self).__init__()
        self.percentage = percentage

    def __call__(self, sample: dict):
        len_data = len(sample['data'][0])
        interval_to_reset = int(len_data * self.percentage)
        power_freq_data = fft(sample['data'])
        for ncanal in range(NVIDEOS):
            begin = random.randint(0, len_data - interval_to_reset)
            power_freq_data[ncanal, begin : begin + interval_to_reset] = 0
        sample['data'] = ifft(power_freq_data)
        return sample
NSUBJECTS = 32
NVIDEOS = 40
NCANALS = 32
       

# class flatten(object):
#     def __init__(self):
#         super(flatten, self).__init__()
        

#     def __call__(self, sample: dict):
#         input = sample['data'].copy()
#         len_seq = len(input)
#         segments = np.array(input).reshape(len_seq, -1)
#         sample['data'] = segments
#         #print(sample)
#         return sample    






class to_scalogram(object):
     def __init__(self):
         super(to_scalogram, self).__init__()
    #  def  matrix_to_scale(self, matrix):
    #      matrix_new = np.zeros((32, 32))
    #      tmax = matrix.shape[1]
    #      for ncanal in range(NCANALS):
    #          coef, freqs=pywt.cwt( matrix[ncanal, :], np.arange(1, 50), 'morl', 1/128)
    #          matrix_new[ncanal, :] = [(coef[s] ** 2).sum()/(-(coef[s]**2/(coef[s] ** 2).sum() * np.log(coef[s]**2/(coef[s] ** 2).sum())).sum()) for s in range(0, 32)]
    #         #  for s in range(7, 38):
    #         #      energy = (coef[s] ** 2).sum()
    #         #      entropy = -(coef[s]**2/energy * np.log(coef[s]**2/energy)).sum()
    #         #      matrix_new[ncanal, s - 7] = energy/entropy
    #      return matrix_new

     def matrix_to_scale(self, matrix):
            matrix_new = np.zeros((32, 32))
            tmax = matrix.shape[1]
            
            for ncanal in range(NCANALS):
                coef, freqs=pywt.cwt( matrix[ncanal, :], np.arange(1, 50), 'morl', 1/128)
                #matrix_new[ncanal, :] = np.zeros()[(coef[s] ** 2).sum()/(-(coef[s]**2/(coef[s] ** 2).sum() * np.log(coef[s]**2/(coef[s] ** 2).sum())).sum()) for s in range(0, 32)]
                #for s in range(0, 32):
                coef = coef[0:32]
                energy = (coef ** 2).sum(axis = 1)
                energy_ratio = coef ** 2/energy.reshape(-1, 1)
                entropy = (-energy_ratio * np.log(energy_ratio)).sum(axis = 1) #-(coef[s]**2/energy * np.log(coef[s]**2/energy)).sum()
                matrix_new[ncanal, :] = energy/entropy
            return matrix_new   
 
     def __call__(self, sample: dict):
         input = sample['data'].copy()
         len_seq = len(input)
         sample['data'] = []
         for i in range(len_seq):
             sample['data'].append(self.matrix_to_scale(input[i]))
         sample['data'] = np.array(sample['data'])
         return sample 

class ToTensor(object):

    def __init__(self, ):
        super(ToTensor, self).__init__()

    def __call__(self, sample: dict):
        return {"labels": torch.tensor(sample["labels"], dtype=torch.long),
                "data": torch.tensor(sample["data"], dtype=torch.float32),
                } 


class EmotionDataset(Dataset):
   def __init__ (self, data, labels, transforms, inds, interval = 1, shift = 1, len_seq = 10):
       self.data = data
       self.labels = labels
       self.transforms = transforms
       self.interval = interval
       self.len_seq = len_seq
       self.cnt = [Counter(), Counter(), Counter(),Counter()]
       self.indexes = []
       for sub in range(len(data)):
            for nvideo in inds[sub]:
               for nsec in range(3, LEN_RECORD + 3, shift):
                  if ((nsec + len_seq * interval) > LEN_RECORD + 3):
                    break
                  self.indexes.append((sub, nvideo, nsec))

            
   def __len__(self):
        result =  len(self.indexes)
        return result

   def __getitem__(self, item):         
       sample = {}   
       (sub, nvideo, nsec) = self.indexes[item]
       sample['labels'] = self.labels[sub][nvideo, :]
       sample['data'] = []
       nsec_it = nsec
       for i in range(self.len_seq):
          sample['data'].append(self.data[sub][nvideo, :32, nsec_it * 128 : (nsec_it + self.interval) * 128])
          nsec_it += self.interval
       
       if self.transforms is not None:
           for t in self.transforms:
                sample = t(sample)
       #print(sample)         
       return sample
       

In [29]:
import glob
import pickle
from sklearn.preprocessing import MinMaxScaler

data = []
labels = []
data_dir = './data_preprocessed_python'
files = glob.glob(os.path.join(data_dir, "*.dat"))
data_raw = []
for file_data in files:
    raw_data = pickle.load(open(file_data, 'rb'), encoding='latin1')
    data.append(raw_data['data'])
    labels.append(raw_data['labels'])
    #break

In [30]:
type_emotion = 0

In [31]:
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import f1_score, accuracy_score
k  = 5
labels_bin = []
for sub in range(32):
  temp = labels[sub] >= 4.5
  #print(labels[i])
  #print(temp)
  labels_bin.append(temp)
  print(sum(labels_bin[sub][:, type_emotion]), end=' ')




20 28 33 30 26 23 24 27 23 20 20 22 17 26 30 26 28 25 22 32 19 24 27 33 25 23 28 29 22 30 17 29 

In [32]:
from sklearn.model_selection import StratifiedKFold, KFold 
from sklearn.metrics import f1_score, accuracy_score
k  = 5
labels_bin = []
for sub in range(32):
  temp = labels[sub] >= 4.5
  #print(labels[i])
  #print(temp)
  labels_bin.append(temp)
  print(sum(labels_bin[sub][:, type_emotion]), end=' ')

inds_train = []
inds_test = []
for sub in range(32):  
    X = np.arange(40)
    y = np.array(labels_bin[sub][:, type_emotion])
    skf = KFold(n_splits=k, random_state=None, shuffle=True)
    balanced_split = skf.split(X, y)
    for ind_train, ind_test in  balanced_split:
        #print(ind_train, ind_test)
        #количестов меток 1 в трейне и тесте
        #print(sum(labels_bin[sub][ind_train, type_emotion]), sum(labels_bin[sub][ind_test, type_emotion]))
        #print((labels_bin[sub][ind_train, type_emotion]), (labels_bin[sub][ind_test, type_emotion]))
        inds_train.append(ind_train)
        inds_test.append(ind_test)

20 28 33 30 26 23 24 27 23 20 20 22 17 26 30 26 28 25 22 32 19 24 27 33 25 23 28 29 22 30 17 29 

In [33]:
writer = None

In [34]:
def train(model, loader, criterion, optimizer, device, val_dataloader, val_f1_min,  description, batch = None, writer = None):
    model.train()
    train_loss = []
    inputs = []
    #torch.autograd.set_detect_anomaly(True)
   
    #lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)#, mode='min', factor=0.1, patience=10, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
    for i , batch in enumerate(tqdm(loader, total=len(loader), desc="training...", position=0 , leave = True)):
            model.train()
            optimizer.zero_grad()
            src  = batch['data'].to(device)
            #print(src)
            trg = batch['labels'][:, 0]
            #print(batch)
            #print(trg)
            #print(src.shape)
            #print(trg.shape)
            levels_pred = model(src)  # B x (2 * NUM_PTS)
           
            #break
            #print(levels_pred.shape)
            levels_pred = levels_pred.cpu()
            #print(levels_pred)
            #break
            #print(levels_pred.shape)
            loss = criterion(levels_pred, trg) 

            # loss = 0
            # #loss_all = []
            
            # for class_value in range(2):
            #       mask = (trg == class_value) 
            #       levels_pred_sub = levels_pred[mask]
            #       trg_sub = trg[mask]
            #       #len += trg.shape[0]
            #       #print(f"{trg.shape[0]}", end = " ")
            #       if (trg_sub.shape[0] != 0):
            #             #loss_all.append(criterion(levels_pred_sub, trg_sub)/trg_sub.shape[0])
            #             loss += criterion(levels_pred_sub, trg_sub)/trg_sub.shape[0] 
            # #loss = loss_all[0] + loss_all[1] + (loss_all[0] - loss_all[1]) ** 2
            train_loss.append(loss.item())
            #print(train_loss[-1])
            loss.backward()
            #print(train_loss[-1])
            optimizer.step()
            if writer:
                writer.add_scalar(f'{description}/training loss per batch',
                                  loss.item(),
                                  i)

            # if (i % 100 == 0):
            #     acc, f1 = (calculate_predictions(model, val_dataloader))
            #     if (f1 > val_f1_min):
            #           val_f1_min      = f1
            #           torch.save({'model_state_dict': model.state_dict(),    'optimizer_state_dict': optimizer.state_dict(),}, os.path.join(args.output_dir, f"val.tgz"))
            #break
    return np.mean(train_loss), val_f1_min


In [35]:
def evaluate(model, loader, criterion, device, writer, description):
    
    model.eval()
    epoch_loss = 0
    history = []
  
    with torch.no_grad():
    
        for s, batch in enumerate(loader):#, total=len(loader), desc="validating...", position=0 , leave = True)):
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, 0]
            #print(src.shape)


            levels_pred = model(src)  # B x (2 * NUM_PTS)
            #print(levels_pred.shape)
            levels_pred = levels_pred.cpu()
            loss = criterion(levels_pred, trg) 
            #print(loss)
            epoch_loss += loss.item() 


            if writer:
                writer.add_scalar(f'{description}/val loss per batch',
                                  loss.item(),
                                  s)
        
    return epoch_loss / s

In [36]:
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

def calculate_predictions(model, loader, output = False):
    model.eval()
    epoch_loss = 0
    history = []
    real = []
    pred = []
    with torch.no_grad():

        for i, batch in enumerate(loader):#, total=len(loader), desc="predicting...", position=0 , leave = True)):
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, 0]
         

            levels_pred = model(src)  # B x (2 * NUM_PTS)
            levels_pred = levels_pred.cpu()
            #print(levels_pred.shape)
            trg_pred = levels_pred.argmax(1)
            
            real.extend(trg)
            pred.extend(trg_pred) 

            
        if output == True:    
            print(accuracy_score(real, pred)) 
            print(confusion_matrix(real, pred))  
            print(classification_report(real, pred))  
        f1 = ((f1_score(real, pred, average = 'binary', pos_label = 0))  + (f1_score(real, pred, average = 'binary', pos_label = 1)))/2
        return (accuracy_score(real, pred)) , f1
        #plt.hist(real)

In [37]:
def train_loop(description, n_epochs = 10):
    #args.epochs = 10
    #criterion =  fnn.mse_loss
    train_loss_min = 10000
    val_f1_min = -10000

    #batch = next(iter(train_dataloader))
    for epoch in range(n_epochs):
          #logger.info(f"Starting epoch {epoch + 1}/{args.epochs}.")
    
          train_loss, val_f1_min  = train(model, train_dataloader, criterion, optimizer ,device, val_dataloader, val_f1_min ,  description , None,  writer)
          #if epoch % 500 == 0:
          if writer:
                writer.add_scalar(f"{description}/training loss per epoch",
                                        train_loss,
                                        epoch)
          print(train_loss)

          if (train_loss < train_loss_min):
                 train_loss_min      = train_loss
                 torch.save({
                         'model_state_dict': model.state_dict(),
                         'optimizer_state_dict': optimizer.state_dict(),
                       },
                       os.path.join(args.output_dir, "train.tgz")
            )  

          val_loss = evaluate(model, val_dataloader, criterion, device,  writer, description )
          # #break
          print(val_loss)
          #if writer:
          #      writer.add_scalar(f"{description}/val loss per epoch",
          #                        val_loss,
          #                        epoch)
          acc, f1 = (calculate_predictions(model, val_dataloader, False))
          print(acc, f1)
          if (acc > val_f1_min):
                       val_f1_min      = acc
                       torch.save({'model_state_dict': model.state_dict(),    'optimizer_state_dict': optimizer.state_dict(),}, os.path.join(args.output_dir, f"val_{description}.tgz")) 

In [38]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [39]:
type_emotion = 0

In [ ]:
import pandas as pd
LEN_RECORD = 60
args.batch_size = 4
k = 4
#transforms_random = RandomAugmentation([add_noise, reset_part_in_freq, reset_part_in_time, None], 0.2)
#transforms = [RandomAugmentation([add_noise(), reset_part_in_freq(0.2), reset_part_in_time(0.2), None], 0.2), to_head_matrix(),ToTensor()]   
transforms = [to_scalogram(),ToTensor()] 
#transforms = [RandomAugmentation([add_noise(), None], 0.5), to_scalogram(),ToTensor()]   
#transforms = [RandomAugmentation([reset_part_in_time(0.4), None], 0.1), to_head_matrix(),ToTensor()]   
acc_all = []
f1_all = []
for sub in range(0, 1): 
    print(f"***sub {sub}****") 
    X = np.arange(40)
    y = np.array(labels_bin[sub][:, type_emotion])
    #skf = StratifiedKFold(n_splits=k, random_state=None, shuffle=True)
    kf = KFold(n_splits=k, random_state=None, shuffle=True)
    #balanced_split = skf.split(X, y)
    balanced_split = kf.split(X)
    acc_sub = []
    f1_sub = []
    for fold, (ind_train, ind_test) in  enumerate(balanced_split):
        #print(ind_train, ind_test)
        print(f"***fold {fold}****") 
        #количестов меток 1 в трейне и тесте
        #print(sum(labels_bin[sub][ind_train, type_emotion]), sum(labels_bin[sub][ind_test, type_emotion]))
        #print((labels_bin[sub][ind_train, type_emotion]), (labels_bin[sub][ind_test, type_emotion]))
        #inds_train = [ 0,  1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 14, 15, 17, 20, 21, 22,  23, 25, 27, 28, 30, 31, 33, 34, 35, 36, 37, 38, 39]
        #inds_test = [ 6, 12, 13, 16, 18, 19, 24, 26, 29, 32]      
        inds_train = []
        inds_test = []
        inds_train.append(ind_train)
        inds_test.append(ind_test)
        print(inds_train)
        print(inds_test)
        print(labels_bin[sub][inds_test, 0])

        train_dataset = EmotionDataset(data[sub : sub + 1 ], labels_bin[sub  : sub + 1], transforms, inds_train, interval = 1, len_seq = 60)
        train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
                                        pin_memory=True, shuffle=True, drop_last=True)

        val_dataset = EmotionDataset(data[sub : sub + 1], labels_bin[ sub : sub + 1], transforms,  inds_test, interval = 1, len_seq = 60)
        val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
                                      pin_memory=True, shuffle=False, drop_last=False)
        print(val_dataset.__len__())
        print(train_dataset.__len__())
        
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        LEN_SEQ = 60
        DROPOUT_COEFF1 = 0
        DROPOUT_COEFF2 = 0
        model = get_model(HIDDEN_SIZE, DROPOUT_COEFF1, DROPOUT_COEFF2, LEN_SEQ).to(device)
        model.apply(initialize_weights)
        criterion = nn.NLLLoss(reduction = 'mean')#torch.nn.MSELoss()
        #optimizer = optim.SGD(model.parameters(), lr=3e-5, momentum = 0.9)#, weight_decay=args.weight_decay)
        #optimizer = optim.RMSprop(model.parameters())#optim.Adam(model.parameters(), lr=3e-4)#, momentum = 0.9)#, weight_decay=args.weight_decay)
        optimizer = optim.Adam(model.parameters(), lr=3e-5)#, momentum = 0.9)#, weight_decay=args.weight_decay)
        train_loop('lstm_subject1',  30)
        description = 'lstm_subject1'
        model_state  = torch.load(os.path.join(args.output_dir, f"val_{description}.tgz"))
        #   #model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device)
        model.load_state_dict(model_state['model_state_dict'])
        acc, f1 = calculate_predictions(model, val_dataloader, True)
        acc_sub.append(acc)
        f1_sub.append(f1)
        print(acc, f1)
    acc_all.append(acc_sub)    
    f1_all.append(f1_sub)  
    #f1_data_1 = pd.DataFrame(f1_all, columns = ['f1_fold1', 'f1_fold2', 'f1_fold3', 'f1_fold4', 'f1_fold5'])
    #acc_data_1 = pd.DataFrame(acc_all, columns = ['acc_fold1', 'acc_fold2', 'acc_fold3', 'acc_fold4', 'acc_fold5'])
    f1_data_1 = pd.DataFrame(f1_all, columns = ['f1_fold1', 'f1_fold2', 'f1_fold3', 'f1_fold4'])
    acc_data_1 = pd.DataFrame(acc_all, columns = ['acc_fold1', 'acc_fold2', 'acc_fold3', 'acc_fold4'])
    
    f1_data_1.to_csv("f1_data_1_lstm222.csv")
    acc_data_1.to_csv("acc_data_1_lstm222.csv")  

***sub 0****
***fold 0****
[array([ 1,  2,  4,  5,  6,  7,  8,  9, 10, 12, 13, 14, 15, 16, 18, 19, 20,
       21, 23, 24, 26, 27, 28, 30, 31, 35, 36, 37, 38, 39])]
[array([ 0,  3, 11, 17, 22, 25, 29, 32, 33, 34])]
[[ True  True False  True  True  True False False False False]]
10
30


training...:   0%|          | 0/7 [00:00<?, ?it/s]

torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
0.7071745906557355
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([2, 2])
1.0819390416145325
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([2, 2])
0.4 0.2857142857142857


training...:   0%|          | 0/7 [00:00<?, ?it/s]

torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
0.6895506637437003
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([2, 2])
1.0823576152324677
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([2, 2])
0.4 0.2857142857142857


...

torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
0.5434872763497489
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([2, 2])
1.0612666010856628
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([2, 2])
0.7 0.6703296703296704


training...:   0%|          | 0/7 [00:00<?, ?it/s]

torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
0.5410545979227338
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([2, 2])
1.0607317090034485
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([2, 2])
0.7 0.6703296703296704


training...:   0%|          | 0/7 [00:00<?, ?it/s]

torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
description = 'lstm_subject1'
model_state  = torch.load(os.path.join(args.output_dir, f"val_{description}.tgz"))
        #   #model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device)
model.load_state_dict(model_state['model_state_dict'])
acc, f1 = calculate_predictions(model, train_dataloader, True)
print(acc, f1)

torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
torch.Size([4, 2])
0.5
[[ 0 14]
 [ 0 14]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           1       0.50      1.00      0.67        14

    accuracy                           0.50        28
   macro avg       0.25      0.50      0.33        28
weighted avg       0.25      0.50      0.33        28

0.5 0.3333333333333333


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
description = 'lstm_subject1'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
LEN_SEQ = 6
DROPOUT_COEFF1 = 0.5
DROPOUT_COEFF2 = 0.5
model = get_model(HIDDEN_SIZE, DROPOUT_COEFF1, DROPOUT_COEFF2, LEN_SEQ)
model_state  = torch.load(os.path.join(args.output_dir, f"val_{description}.tgz"))
#   #model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device)
model.load_state_dict(model_state['model_state_dict'])
acc, f1 = calculate_predictions(model, val_dataloader, True)
print(calculate_predictions(model, val_dataloader, True))

0.5
[[0 5]
 [0 5]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.50      1.00      0.67         5

    accuracy                           0.50        10
   macro avg       0.25      0.50      0.33        10
weighted avg       0.25      0.50      0.33        10



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


KeyboardInterrupt: ignored

In [ ]:
print(val_dataset.__len__())

In [ ]:

args.batch_size = 4
k = 3
#transforms_random = RandomAugmentation([add_noise, reset_part_in_freq, reset_part_in_time, None], 0.2)
#transforms = [RandomAugmentation([add_noise(), reset_part_in_freq(0.2), reset_part_in_time(0.2), None], 0.2), to_head_matrix(),ToTensor()]   
transforms = [to_segments(),ToTensor()] 
#transforms = [RandomAugmentation([add_noise(), None], 0.5), to_head_matrix(),ToTensor()]   
#transforms = [RandomAugmentation([reset_part_in_time(0.4), None], 0.1), to_head_matrix(),ToTensor()]   
acc_all = []
f1_all = []
for sub in range(1, 2): 
    print(f"***sub {sub}****") 
    X = np.arange(40)
    y = np.array(labels_bin[sub][:, type_emotion])
    skf = StratifiedKFold(n_splits=k, random_state=None, shuffle=True)
    balanced_split = skf.split(X, y)
    acc_sub = []
    f1_sub = []
    for fold, (ind_train, ind_test) in  enumerate(balanced_split):
        #print(ind_train, ind_test)
        print(f"***fold {fold}****") 
        #количестов меток 1 в трейне и тесте
        #print(sum(labels_bin[sub][ind_train, type_emotion]), sum(labels_bin[sub][ind_test, type_emotion]))
        #print((labels_bin[sub][ind_train, type_emotion]), (labels_bin[sub][ind_test, type_emotion]))
        inds_train = []
        inds_test = []
        inds_val = []

        ind_val = random.sample(list(ind_test), 2)
        ind_test = list(set(ind_test).difference(ind_val))
        print(ind_train, ind_test, ind_val)
        inds_train.append(ind_train)
        inds_test.append(ind_test)
        inds_val.append(ind_val)

        print(inds_train)
        print(inds_test)
        print(labels_bin[sub][inds_test, 0])

        train_dataset = EmotionDataset(data[sub : sub + 1 ], labels_bin[sub  : sub + 1], transforms, inds_train)
        train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
                                        pin_memory=True, shuffle=True, drop_last=True)

        val_dataset = EmotionDataset(data[sub : sub + 1], labels_bin[ sub : sub + 1], transforms,  inds_test)
        val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
                                      pin_memory=True, shuffle=False, drop_last=False)
        

        test_dataset = EmotionDataset(data[sub : sub + 1], labels_bin[ sub : sub + 1], transforms,  inds_val)
        test_dataloader = DataLoader(test_dataset, batch_size=2, num_workers=1,
                                      pin_memory=True, shuffle=False, drop_last=False)
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model = get_model()
        model.apply(initialize_weights)
        criterion = nn.CrossEntropyLoss(reduction = 'mean')#torch.nn.MSELoss()
        #optimizer = optim.SGD(model.parameters(), lr=3e-5, momentum = 0.9)#, weight_decay=args.weight_decay)
        optimizer = optim.Adam(model.parameters(), lr=3e-4)#, momentum = 0.9)#, weight_decay=args.weight_decay)
        train_loop('lstm_subject1',  20)
        description = 'lstm_subject1'
        model_state  = torch.load(os.path.join(args.output_dir, f"val_{description}.tgz"))
        #   #model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device)
        model.load_state_dict(model_state['model_state_dict'])
        acc, f1 = calculate_predictions(model, test_dataloader, True)
        acc_sub.append(acc)
        f1_sub.append(f1)
        print(acc, f1)
    acc_all.append(acc_sub)    
    f1_all.append(f1_sub)  
    #f1_data_1 = pd.DataFrame(f1_all, columns = ['f1_fold1', 'f1_fold2', 'f1_fold3', 'f1_fold4', 'f1_fold5'])
    #acc_data_1 = pd.DataFrame(acc_all, columns = ['acc_fold1', 'acc_fold2', 'acc_fold3', 'acc_fold4', 'acc_fold5'])
    f1_data_1 = pd.DataFrame(f1_all, columns = ['f1_fold1', 'f1_fold2', 'f1_fold3', 'f1_fold4'])
    acc_data_1 = pd.DataFrame(acc_all, columns = ['acc_fold1', 'acc_fold2', 'acc_fold3', 'acc_fold4'])
    
    f1_data_1.to_csv("f1_data_1_lstm222.csv")
    acc_data_1.to_csv("acc_data_1_lstm222.csv")  

NameError: ignored

In [ ]:
 def  matrix_to_scale(self, matrix):
         matrix_new = np.zeros((32, 32))
         tmax = matrix.shape[1]
         for ncanal in range(NCANALS):
             coef, freqs=pywt.cwt( matrix[ncanal, :], np.arange(1, 50), 'morl', 1/128)
             matrix_new[ncanal, :] = [(coef[s] ** 2).sum()/(-(coef[s]**2/(coef[s] ** 2).sum() * np.log(coef[s]**2/(coef[s] ** 2).sum())).sum()) for s in range(0, 32)]
            #  for s in range(7, 38):
            #      energy = (coef[s] ** 2).sum()
            #      entropy = -(coef[s]**2/energy * np.log(coef[s]**2/energy)).sum()
            #      matrix_new[ncanal, s - 7] = energy/entropy
         return matrix_new

In [ ]:
import pickle
videos = []
for i in range(40):
  videos.append([]) 
  for nsec in range(60):
     matrix = matrix_to_scale(data[0][nvideo, :32, (3 + nsec) * 128 : (4 + nsec) * 128 ])
     videos[i].append(matrix[np.newaxis, :, :])
     
